# GeekBrains, ML in Business
# Lesson 7 Homework

Ссылки:
- Пример с Permutation Importance - https://www.kaggle.com/dansbecker/permutation-importance
- Github проекта SHAP - https://github.com/slundberg/shap
- https://arxiv.org/pdf/1809.04559.pdf
- https://medium.com/civis-analytics/demystifying-black-box-models-with-shap-value-analysis-3e20b536fc80
- https://en.wikipedia.org/wiki/Shapley_value

**Импорт библиотек**

In [1]:
import numpy as np
import pandas as pd

import catboost as ctb

import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score


# load JS visualization code to notebook
shap.initjs()

**Классы и функции для задания**

In [2]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """
    Обучить и оценить модель.
    """
    model = ctb.CatBoostClassifier(cat_features=cat_feats)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='binary')
    rec = recall_score(y_test, y_pred, average='binary')
    
    return {'f1': [f1], 'roc-auc': [roc], 'precision': [prec], 'recall': [rec]}

## Задание 1
Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php).

### Решение Задания 1

Kaggle Dataset - HR Analytics: Job Change of Data Scientists:
- https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists

In [5]:
df = pd.read_csv("data/aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [6]:
df.shape

(19158, 14)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Ненужный признак

In [8]:
df = df.drop(columns=['enrollee_id'])

Конвертируем таргет в int

In [9]:
df['target'] = df['target'].astype(int)

Смотрим баланс таргета

In [10]:
df['target'].value_counts()

0    14381
1     4777
Name: target, dtype: int64

## Задание 2
Сделать обзорный анализ выбранного датасета.

### Решение Задания 2

## Задание 3
Сделать feature engineering.

### Решение Задания 3

Просто заменяем все на самое частое значение (моду), т.к. все признаки с пропущенными значениями категориальные.

In [11]:
for col in df.select_dtypes('object').columns:
    df[col] = df[col].fillna(df[col].value_counts().index[0])

Делим данные на трейн и тест.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), df['target'], test_size=0.2, random_state=42)

## Задание 4
Обучить любой классификатор (какой вам нравится).

### Решение Задания 4

In [13]:
cat_feats = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level',
             'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

In [15]:
metrics = pd.DataFrame(evaluate_model(ctb.CatBoostClassifier(cat_features=cat_feats),
                                      X_train,
                                      y_train,
                                      X_test,
                                      y_test))

In [14]:
metrics

,f1,roc-auc,precision,recall
0,0.504113,0.669586,0.572,0.45063


## Задание 5
Разобраться с SHAP и построить важности признаков для:
1. Всего тестового набора данных (summary_plot - дать интерпретацию),
2. Для топ 10%,
3. Для отдельных наблюдений вывести force_plot и попытаться проинтерпретировать результат.

### Решение Задания 5.1

### Решение Задания 5.2

### Решение Задания 5.3